# Определение токсичных комментариев.

Интернет-магазин  запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

**Нужно:**
- Обучить модель классификации комментариев на позитивные и негативные.

**Требования**
 - значение метрики *F1* не меньше 0.75. 

**Описание данных:**
Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Импорт библиотек

In [45]:
import pandas as pd
import re
#from pymystem3 import Mystem
import tqdm

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

import nltk 
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

#отслеживание прогресса
from tqdm.notebook import tqdm
from IPython.display import display
import time

In [46]:
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

Загрузка датасета и просмотр нескольких записей в нём для того-чтобы убедится в корректности загрузки и первично ознакомится с данными.

In [48]:
df = pd.read_csv('toxic_comments.csv', index_col='Unnamed: 0')
df.sample(20)

,text,toxic
6752,"Yachts \n\nOn a decidedly trivial note, I read...",0
31863,"""\n\n I agree with first suggestion his neutr...",0
74257,""", 1 January 2008 (UTC)\n\nThanks for asking. ...",0
94139,No vandalism. Simply removing my original cont...,0
32334,"""\nRead what he said. """"""""If someone talks of ...",0
100801,"Ban expired. Now what, moron -P",1
29155,Tripolis Massacre\n\nHecrtoian. Your sick fana...,1
95913,Probably another station named Ridgewood. I'm ...,0
102711,"""\n\nI have reported User:Tedickey for spam. H...",0
16815,was vandilising come to my house and ill show ...,0


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [50]:
df = df.reset_index(drop=True)

Пропусков нет. Посмотрю на соотношение значений целевого признака.

In [52]:
df['toxic'].value_counts()/df.shape[0]*100

0    89.838787
1    10.161213
Name: toxic, dtype: float64

### Векторизация

Создам небольшой список текстов для тестирования.

In [55]:
test_text = df['text'].iloc[:3]

Создам класс для лемматизации текстов

Создам функцию подготавлювающую текст к лемматизации.

In [58]:
def text_prep(txt):
    return re.sub(r'[^a-zA-Z\']', ' ', txt.lower()).split()
    #return ' '.join(re.sub(r'[^a-zA-Z\']', ' ', txt.lower()).split()).strip().split() 
    #return ' '.join(re.sub(r'[^a-zA-Z\']', ' ', ' '.join(nltk.word_tokenize(txt.lower())) ).split())
text_prep(df['text'].iloc[1])


["d'aww",
 'he',
 'matches',
 'this',
 'background',
 'colour',
 "i'm",
 'seemingly',
 'stuck',
 'with',
 'thanks',
 'talk',
 'january',
 'utc']

Использую функцию для получения POS-тэга. 

In [60]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


Напишу класс-лемматизатор

In [62]:
class Lemmatizer(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
        
    @staticmethod    
    def text_prep(txt):
        return re.sub(r'[^a-zA-Z\']', ' ', txt.lower()).split()
    
    @staticmethod 
    def get_wordnet_pos(word):
        """Map POS tag to first character lemmatize() accepts"""
        tag = nltk.pos_tag([word])[0][1][0].upper()
        tag_dict = {"J": wordnet.ADJ,
                    "N": wordnet.NOUN,
                    "V": wordnet.VERB,
                    "R": wordnet.ADV}
        return tag_dict.get(tag, wordnet.NOUN)
    
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return [ ' '.join(
            [self.lemmatizer.lemmatize( word, self.get_wordnet_pos(word)) for word in self.text_prep(x)]
        )
                for x in tqdm(X, 'lemmatization')]

In [63]:
lemmatizer = Lemmatizer()

In [64]:
lemmatizer.transform(df['text'].iloc[:3])

lemmatization:   0%|          | 0/3 [00:00<?, ?it/s]

["explanation why the edits make under my username hardcore metallica fan be revert they weren't vandalism just closure on some gas after i vote at new york doll fac and please don't remove the template from the talk page since i'm retire now",
 "d'aww he match this background colour i'm seemingly stuck with thanks talk january utc",
 "hey man i'm really not try to edit war it's just that this guy be constantly remove relevant information and talk to me through edits instead of my talk page he seem to care more about the format than the actual info"]

Данный класс будет использован в пайплайне.

### Создание мешка слов. Стоп слова

In [67]:
stop_words = set(stopwords.words('english'))

In [68]:
class BowCreator(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.stop_words = list(stopwords.words('english'))
        self.count_vect = CountVectorizer(stop_words=self.stop_words)
        
    def fit(self, X, y=None):
        t = time.time()
        display('count_vect fit start')
        self.count_vect.fit(X)
        t = time.time() - t
        display(f'count_vect fit end [{t:.3f}]sec.')
        return self
    
    def transform(self, X):
    # X - лемматизированные тексты
        display('bow creation process starting')
        t = time.time()
        bow = self.count_vect.transform(X)
        t = time.time() - t
        display(f'bow creation process ended [{t:.3f}]sec.')
        display(f'shape {bow.shape}')
        return bow

In [69]:
bow_creator = BowCreator()

In [70]:
bow_creator.fit_transform(lemmatizer.fit_transform(test_text)).toarray()

lemmatization:   0%|          | 0/3 [00:00<?, ?it/s]

'count_vect fit start'

'count_vect fit end [0.002]sec.'

'bow creation process starting'

'bow creation process ended [0.000]sec.'

'shape (3, 48)'

array([[0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0,
        1, 1, 0, 1],
       [0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1,
        0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0,
        0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 2, 0, 0, 1, 0, 0,
        0, 0, 1, 0]], dtype=int64)

Данный класс также будет использоваться в пайплайне.

## Обучение

### Создание выборок

In [74]:
RANDOM_STATE = 0
X_train,X_test, y_train, y_test = train_test_split(
    df['text'],
    df['toxic'],
    shuffle=False,
    random_state=RANDOM_STATE,
    test_size = .4
)

X_valid, X_test, y_valid, y_test = train_test_split(
    X_test,
    y_test,
    shuffle=False,
    random_state=RANDOM_STATE,
    test_size = .5
)

### Создание пайплайна
Данные преобразую заранее так-как этот процесс занимает очень много времени.

In [76]:
pipe = Pipeline([
    ('lemmatizer', Lemmatizer()),
    ('vectorizer', BowCreator())
])

In [77]:
X_train = pipe.fit_transform(X_train)
X_valid = pipe.transform(X_valid)
X_test = pipe.transform(X_test)

lemmatization:   0%|          | 0/95575 [00:00<?, ?it/s]

'count_vect fit start'

'count_vect fit end [4.295]sec.'

'bow creation process starting'

'bow creation process ended [4.112]sec.'

...

'shape (31858, 112107)'

lemmatization:   0%|          | 0/31859 [00:00<?, ?it/s]

'bow creation process starting'

'bow creation process ended [1.389]sec.'

'shape (31859, 112107)'

### Обучение модели

#### LogisticRegression

In [80]:
%%time
linear = LogisticRegression(max_iter=1000)
linear.fit(X_train, y_train)
preds = linear.predict(X_valid)
print(f'Linear f1: {(score_linear := f1_score(y_valid, preds)):.2%}')

Linear f1: 75.41%
CPU times: total: 12.8 s
Wall time: 12.9 s


#### RidgeClassifier

In [82]:
%%time
ridge = RidgeClassifier(alpha=1, max_iter=1000)
ridge.fit(X_train, y_train)
preds = ridge.predict(X_valid)
print(f'Linear with l2 f1: {(score_ridge := f1_score(y_valid, preds)):.2%}')

Linear with l2 f1: 43.68%
CPU times: total: 49.4 s
Wall time: 48.5 s


#### Tree

In [84]:
%%time
tree = DecisionTreeClassifier(random_state=RANDOM_STATE)
tree.fit(X_train, y_train)
preds = tree.predict(X_valid)
print(f'Decision Tree f1: {(score_tree := f1_score(y_valid, preds)):.2%}')

Decision Tree f1: 69.44%
CPU times: total: 2min 36s
Wall time: 2min 38s


### Проверка лучшей модели на тестовых данных

In [86]:
preds = linear.predict(X_test)
print(f'Linear f1: {(score_linear := f1_score(y_test, preds)):.2%}')

Linear f1: 75.35%


## Выводы

Для выполнения работы по обучению модели классификации отзывов были использованны предварительно размеченные данные. Эти данные были лемматизированы с помощью инструментов библиотеки `nltk`, так-как это одна из библиотек которая моддерживает лемматизацию англоязычных текстов, далее эти тексты были векторизированы с удалением стоп-слов.   
Полученный набор данных был использован при обучении моделей, из которых была выбрана лучшая.  
Лучшая модель показала значения метрики на тестовых данных удовлетворяющие условиям.